In [1]:
import numpy as np
import pandas as pd
import MDAnalysis as mda
import os
from numba import jit
import time
from matplotlib import pyplot as plt
from scipy.stats import linregress

In [2]:
#Section1: Data Loading
print("Data Loading")
start=time.time()

u=mda.Universe("dump.lammpstrj", format="LAMMPSDUMP", dt=5)
n_atoms=len(u.atoms)
n_frames=len(u.trajectory)
pos=np.zeros((n_frames, n_atoms, 3))
u.atoms.masses=1.0

for i,ts in enumerate(u.trajectory): #using ts and u.trajectory: To read all the atoms (n_atoms) in a given frame
    pos[i]=u.atoms.positions

end=time.time()
t1=end-start
print(f"Time Taken={end-start:.3f}sec")

Data Loading


/opt/anaconda3/lib/python3.13/site-packages/MDAnalysis/topology/LAMMPSParser.py:670: UserWarning: Guessed all Masses to 1.0
  warnings.warn("Guessed all Masses to 1.0")


Time Taken=13.662sec


In [3]:
#Section2: MSD Definition
print("Parameters and MSD definition")
start=time.time()

dt=0.005
dump_freq=1000 #Storing coordinates at every 1000th step
t_bw_frames=dt*dump_freq #Time between two frames

#First 10^5 steps are for eqbr, so leave them
t_skip=dt*(10**5)
n_frame_skip=int(t_skip/t_bw_frames) #Number of frames to be skipped
n_blocks=5 #We'll use 5 blocks to find Diffusivity
trajfrac=0.5 #States that our lagtime would be "upto" 50% of trajectory (can go up to 50%, not a single value)

#MSD Function
@jit(nopython=True) #nopython=True makes sure that if jit doesn't work, we'll get an error, and we won't be using the slow Python
def MSD(pos, trajfrac):
    n_frames=pos.shape[0]
    max_lag=int(trajfrac*n_frames) #lag in terms of the number of frames: lag defines the time stretch between two data points
    msd_list=[]

    for i in range(1, max_lag):
        dist=pos[i:-(max_lag-i)]-pos[0:-max_lag]
        sq_dist=np.square(dist)
        mean_sq=np.mean(np.sum(sq_dist, axis=2)) #axis=2 is the coordinate axis
        msd_list.append(mean_sq)
    return np.array(msd_list)

end=time.time()
t2=end-start
print(f"Time Taken={end-start:.3f}sec")

Parameters and MSD definition
Time Taken=0.003sec


In [4]:
#Section3: Calculations
print("Calculations")
start=time.time()

use_data=pos[n_frame_skip:]
n_frames_per_block=int(use_data.shape[0]/n_blocks)
block_msd=[]

for i in range(n_blocks):
    frame_start=i*n_frames_per_block
    frame_end=frame_start+n_frames_per_block
    block_data=use_data[frame_start:frame_end]
    res=MSD(block_data, trajfrac)
    block_msd.append(res)

block_msd=np.array(block_msd) #For math operations, better to use numpy array

avg_msd=np.mean(block_msd, axis=0)
sde_msd=np.mean(block_msd, axis=0)

lag_times=np.arange(1, len(avg_msd)+1)*t_bw_frames

end=time.time()
t3=end-start
print(f"Time Taken={end-start:.3f}sec")

Calculations
Time Taken=6.657sec


In [5]:
#Section4: Diffusivity
print("Final Results")
start=time.time()

#Theoretical Value of diffusivity (as per the Chapman-Enskog Theory)
#D=3/8 root(pi*m*k_B*T)/(pi*sigma^2*omega_(1,1)*rho): This theory works for binary collisions, uncorr vel of colliding molecules
#omega_(1, 1): For LJ i/a(s) only
#sigma: Collision diameter

#For dimensionalizing, we'll use the temp scale factor (eps/kB) and length scale factor (sigma) of Ar described by LJ
#rho_scale=mass of 1 atom/sigma^3: 6*10^23 atoms weigh 40g, so rho_scale=40*10^(-3)/(6*10^23)/sigma^3
#We can directly make use of Diffusivity values provided by Edward B Winn in his 1950 paper: The Temperature Dependence of the Self-Diffusion Coefficients of Argon, Neon, Nitrogen, Oxygen, Carbon Dioxide, and Methane*

def dimen_diff(D):
    sigma=3.4*(10**(-10))
    tau=2.15*(10**(-12))
    fac=sigma**2/tau
    D_dim=fac*D
    return D_dim

start_fit=int(0.4*len(lag_times))
slope, intercept, r_value, p_value, sde=linregress(lag_times[start_fit:], avg_msd[start_fit:])

#For our 3-D case, MSD=6Dt
D=slope/6
D_dim=dimen_diff(D)
print(f"Calculated Diffusivity={D_dim:.3e}m²/s")
print(f"Literature Value={0.018/10**4:.3e}m²/s")
print(f"R² value={r_value**2:.3f}")

end=time.time()
t4=end-start
print(f"Time Taken={end-start:.3f}sec")
print(f"Total Time Taken={t1+t2+t3+t4:.3f}")

Final Results
Calculated Diffusivity=5.340e-06m²/s
Literature Value=1.800e-06m²/s
R² value=0.999
Time Taken=0.002sec
Total Time Taken=20.325
